<a href="https://colab.research.google.com/github/jakansha2001/CoVShorts-News-Summarization-Application-Based-on-Deep-NLP-Transformers-for-SARS-CoV-2/blob/master/Original_Dataset_Preprocessing_Bart_T5_300_transformers4_5_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BART AND T5

#NLP model Functions

In [ ]:
!pip install transformers==4.5.0
!pip install sentencepiece

In [ ]:
# !pip install spacy
!pip install spacy

T5

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def t5Sum(body):
  tokens_input = tokenizer.encode(body, return_tensors="pt", max_length=512, truncation=True)
  summary_ids = model.generate(tokens_input, min_length=60, max_length=180, length_penalty=4.0)
  summary = tokenizer.decode(summary_ids[0])
  return summary

BART

In [ ]:
from transformers import pipeline
bartSummarizer = pipeline('summarization', model='facebook/bart-large-cnn', tokenizer='facebook/bart-large-cnn')

def bartSum(body):
  summary=bartSummarizer(body, min_length = round(0.1 * len(body.split(' '))), max_length = round(0.2 * len(body.split(' '))), do_sample=False, truncation=True)
  return summary[0]['summary_text']

#Importing Covid article dataset csv file



In [ ]:
#from google.colab import drive
#drive.mount("/content/gdrive")

In [ ]:
import pandas as pd

In [ ]:
#df=pd.read_csv('/content/gdrive/My Drive/covid-19dataset/covid_dataset.csv')

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/jakansha2001/Datasets/master/covid19_articles1000.csv')

In [ ]:
#dataset=df.values.tolist()
dataset=df['content'].tolist()

In [ ]:
small_dataset = dataset[300:400]

Preprocessing the Data

In [ ]:
import re

In [ ]:
def preprocessor(text):
  text = text.lower()
  text = re.sub(re.compile('<.*?>'),'',text)
  text = re.sub('\[.*?\]','',text)
  #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  #text = re.sub('[''""...]','',text)
  text = re.sub('\n','',text)
  #text = re.sub('\w*\w*','',text)
  text = re.sub('\xa0','',text)
  text = re.sub('\u200b','',text)
  text = re.sub('http.*?','',text)
  text = re.sub(r'\\','',text)
  return text

processedArticles = []

for article in small_dataset:
    processedArticles.append(preprocessor(article))


Generating summaries

In [ ]:
t5Result = []
bartResult = []

In [ ]:
i = 1
for article in processedArticles:
  t5Result.append(t5Sum(article))
  bartResult.append(bartSum(article))
  print(i)
  i = i + 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
def resultprocessor(text):
  text = re.sub(r'<pad>','',text)
  text = re.sub(r'<extra_id_0>','',text)
  text = re.sub(r'</s>','',text)
  return text

t5ProcessedResult = []

for res in t5Result:
  t5ProcessedResult.append(resultprocessor(res))

In [ ]:
data={'t5-summary':t5ProcessedResult,'bart-summary':bartResult}
df = pd.DataFrame(data)
df.to_csv('bbc_result.csv', index=False)  

In [ ]:
df.head()

,t5-summary,bart-summary
0,trump said the u.s. has the outbreak of the ...,Donald trump says he's not worried about the c...
1,sars outbreak in china has led to concerns ov...,There are 440 confirmed cases of coronavirus i...
2,on wednesday as it reported strong sales in ...,burberry raised its revenue outlook for the y...
3,"virus first detected in wuhan, china, last mo...","Virus was first detected in wuhan, china, last..."
4,u.s. global jets etf jets fell about 3% on j...,Asian stocks were hit hard as travel and comme...


#ROGUE AND BLEU SCORE CALCULATION

In [ ]:
!pip install rouge-score

In [ ]:
t5Rouge = []
bartRouge = []

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [ ]:
for i in range(0,100):
  t5Rouge.append(scorer.score(processedArticles[i], t5ProcessedResult[i]))
  bartRouge.append(scorer.score(processedArticles[i], bartResult[i]))

In [ ]:
print('T5 Rouge Scores: ', t5Rouge)
print('BART Rouge Scores: ', bartRouge)

T5 Rouge Scores:  [{'rouge1': Score(precision=0.926829268292683, recall=0.36538461538461536, fmeasure=0.5241379310344828), 'rougeL': Score(precision=0.7804878048780488, recall=0.3076923076923077, fmeasure=0.44137931034482764)}, {'rouge1': Score(precision=0.9433962264150944, recall=0.06188118811881188, fmeasure=0.11614401858304296), 'rougeL': Score(precision=0.8301886792452831, recall=0.054455445544554455, fmeasure=0.10220673635307781)}, {'rouge1': Score(precision=1.0, recall=0.06055363321799308, fmeasure=0.11419249592169657), 'rougeL': Score(precision=0.9714285714285714, recall=0.058823529411764705, fmeasure=0.11092985318107668)}, {'rouge1': Score(precision=1.0, recall=0.03292181069958848, fmeasure=0.06374501992031872), 'rougeL': Score(precision=0.9821428571428571, recall=0.03233392122281011, fmeasure=0.06260671599317018)}, {'rouge1': Score(precision=0.9375, recall=0.08, fmeasure=0.14742014742014742), 'rougeL': Score(precision=0.796875, recall=0.068, fmeasure=0.12530712530712532)}, {'r

Comparing the Fmeasure of all three models

In [ ]:
t5Fmeasure = []
bartFmeasure = []

In [ ]:
for i in range(0,100):
  t5Fmeasure.append(t5Rouge[i]['rougeL'].fmeasure)
  bartFmeasure.append(bartRouge[i]['rougeL'].fmeasure)

In [ ]:
from statistics import mean

print("Cumulative F Measure")
print("T5 : ", round(mean(t5Fmeasure), 6))
print("BART : ", round(mean(bartFmeasure), 6))

Cumulative F Measure
T5 :  0.14217
BART :  0.173502


In [ ]:
#from nltk.translate.bleu_score import sentence_bleu
#from nltk.translate.bleu_score import SmoothingFunction
#smoothie = SmoothingFunction().method4
#reference = str('''Eight people are dead following two shootings at shisha bars in the western German city of Hanau. At least five people were injured after gunmen opened fire at about 22:00 local time (21:00 GMT), police told the BBC. Police added that they are searching for the suspects, who fled the scene and are currently at large. The first shooting was at a bar in the city centre, while the second was in Hanau's Kesselstadt neighbourhood, according to local reports. Police officers and helicopters are patrolling both areas. An unknown number of gunmen killed three people at the first shisha bar, Midnight, before driving to the Arena Bar & Cafe and shooting dead another five victims, regional broadcaster Hessenschau reports. A dark-coloured vehicle was then seen leaving the scene.The motive for the attack is unclear, a police statement said. Can-Luca Frisenna, who works at a kiosk at the scene of one of the shootings said his father and brother were in the area when the attack took place. It's like being in a film, it's like a bad joke, that someone is playing a joke on us, he told Reuters.I can't grasp yet everything that has happened. My colleagues, all my colleagues, they are like my family - they can't understand it either. Hanau, in the state of Hessen, is about 25km (15 miles) east of Frankfurt. It comes four days after another shooting in Berlin, near a Turkish comedy show at the Tempodrom concert venue, which killed one person.''').split()
#candidate = str('''Eight people are dead following two shootings at shisha bars in the western German city of Hanau. The first shooting was at a bar in the city centre, while the second was in Hanau's Kesselstadt neighbourhood, according to local reports. It's like being in a film, it's like a bad joke, that someone is playing a joke on us, he told Reuters.''').split()
#score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
#print(round(score,6))